In [ ]:
from feat import Detector # py-feat
import imageio.v2 as imageio # to create video from frames
import io

In [ ]:
detector = Detector(face_model='retinaface', 
                    landmark_model='mobilenet', 
                    au_model='xgb', 
                    emotion_model='resmasknet', 
                    facepose_model='img2pose-c', # constrained version of img2pose, for frontal face
                    identity_model='facenet', 
                    device='cuda', 
                    n_jobs=1, 
                    verbose=False,
                    )

In [ ]:

video_path = "F:\\RAVDESS\\Actor_14\\01-01-02-02-02-02-14.mp4"
print(video_path)
output_filename = f'F:\\tracked_overlay\\{video_path[-24:-4]}_tracked.mp4'
print(output_filename)

In [ ]:
print("running py-feat detector")
# run detector
video_prediction = detector.detect_video(video_path, 
                                         skip_frames=None, 
                                         output_size=720, 
                                         batch_size=1, 
                                         num_workers=0, 
                                         pin_memory=False, 
                                         face_detection_threshold=0.5, 
                                         face_identity_threshold=0.8
                                         )


In [ ]:

# Constants for video settings
VIDEO_WIDTH, VIDEO_HEIGHT = 1280, 720  # Desired resolutionFalse
DPI = 100  # Resolution in dots per inch

print("generating overlay for each frame")
# generate overlay for each frame
figs = video_prediction.plot_detections(faces='landmarks', 
                                          faceboxes=True, 
                                          muscles=False, 
                                          poses=True, 
                                          gazes=False, 
                                          add_titles=False, 
                                          au_barplot=False, 
                                          emotion_barplot=False, 
                                          plot_original_image=True
                                          );

# Create a video writer object, specifying the output file, codec, and framerate
writer = imageio.get_writer(output_filename, fps=30, codec='libx264', format='FFMPEG', macro_block_size=None)

print("generating video")
# Loop through each figure in the list, adjust size, save it to a buffer, and append it to the video
for fig in figs:
    # Set the figure size to correspond to the desired pixel dimensions
    # fig.set_size_inches(VIDEO_WIDTH / DPI, VIDEO_HEIGHT / DPI)
    
    # Use a buffer to save the figure
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=DPI)
    buf.seek(0)
    
    # Read the image from the buffer and add it to the video
    image = imageio.imread(buf)
    writer.append_data(image)
    
    # Close the buffer
    buf.close()

# Close the writer to finalize the video
writer.close()

print(f'Video saved as {output_filename}')
